In [ ]:
# import os
# import sys
# print(os.getcwd())

from contextlib import suppress
from http.client import HTTPConnection
import logging

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from c8y_tk.interactive import CumulocityContext

## Intializing the Cumulocity context

A (temporarily) authenticated connection to Cumulocity can be obtained best using the `CumulocityContext` class which
- uses standard `C8Y_*` environment variables when provided
- interactively queries for additional necessary information, e.g. the user's password of current TFA token

In [ ]:
with CumulocityContext() as c8y:
    cu = c8y.users.get_current()
    print(f"Current user's full name: {cu.first_name} {cu.last_name}")

When using Jupyter, a context manager might not be practical, but the `CumulocityContext` class can be normally instantiated as well. Then the connection can be reused in subsequent cells:

In [ ]:
c8y = CumulocityContext()

## Querying data

The Cumulocity Python API is not only for implementing microservices. Its user-friendly design and full code completion support makes it fun to used in interactive session as well. This works in Jupyter notebooks as well (as you might have guessed - looking at such a notebook).

In [ ]:
print("Last registered devices:")
for d in c8y.device_inventory.select(query="$orderby=creationTime desc", limit=20):
    serial = None
    with suppress(AttributeError):
        serial = d.c8y_Hardware.serialNumber
    print(f" - {d.name} (#{d.id}), Serial: {serial or '-'}, Registered: {d.creation_time}")

## Working with measurements

Let's check whether there are some measurements we can work with. First we get the latest measurements and check which devices were involved:

In [ ]:
ms = c8y.measurements.get_all(before='now', reverse=True, limit=1000)
device_ids = [m.source for m in ms]

print("Number of measurements per device:")
print(pd.Series(device_ids).value_counts())

fig = sns.countplot(device_ids)
plt.title("Most active devices")
plt.xlabel("/1000 measurements")
plt.show()

We choose the most active device and plot the measurements of the same kind:

In [ ]:
top_device = pd.Series(device_ids).value_counts().index[0]
print(f"Most active device: {top_device}")

def get_series(m):
    return [f'{k1}.{k2}' for k1, v1 in m.fragments.items() for k2 in v1.keys()]

# get defined series names for found measurements
device_ms = [m for m in ms if m.source == top_device]
device_series = [get_series(m)[0] for m in device_ms]

top_series = pd.Series(device_series).value_counts().index[0]
print(f"Most used series: {top_series}")

## Visualizing data

Let's put this into a diagram:

In [ ]:
category, series = top_series.split('.')

# get max 100 values from this series
values = [(m.datetime, m[category][series]['value']) for i, m in enumerate(device_ms) if category in m and series in m[category] and i < 100]
values_df = pd.DataFrame(values, columns=['date', 'value'])
values_df.head()

In [ ]:
sns.pointplot(data=values_df, x='date', y='value')
plt.title(top_series)
plt.xticks(ticks=[], labels=[])
plt.show()